In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import glob
import subprocess

from os import path

np.seterr(all='raise')

def metis_partition_file_converter(inp, out):
    import sys

    partitions = {}

    def add(el, idx):
        if idx not in partitions:
            partitions[idx] = [el]
        else:
            partitions[idx].append(el)


    with open(inp, "r+") as f:
        el = 0
        for line in f:
            idx = int(line.strip())
            add(el, idx)
            el += 1

    with open(out, "w+") as f:
        idxs = list(partitions.keys())
        idxs.sort()
        assert(idxs[0] <= idxs[-1])
        for ix in idxs:
            f.write(" ".join([str(e) for e in partitions[ix]]) + "\n")


import numpy as np


def graph_and_cut_to_numpy(gf, cf):

    graph = {}
    with open(gf, "r") as f:
        first_row = f.readline().strip("\n").split(" ")
        while [] in first_row:
            first_row.remove([])
        print("n, e", first_row)
        n, e = [int(e) for e in first_row]
        for i, row in enumerate(f.readlines()):
            graph[i + 1] = []
            for v in row.strip("\n").split(" "):
                if v == "":
                    continue
                assert(int(v))
                graph[i + 1].append(int(v))
                try:
                    graph[int(v)].append(int(i + 1))
                except KeyError:
                    graph[int(v)] = [int(i + 1)]

    clusters = []
    #TOFIX: clusters are 0 index
    if cf:
        with open(cf, "r") as f:
            n_counter = 0
            e_counter = 0
            for i, row in enumerate(f.readlines()):
                clusters.append([int(e) + 1 for e in row.strip("\n").split(" ") if e != ""])
                #print(clusters[-1])
                n_counter += 1
            e_counter += len(clusters[-1])
    else:
        clusters = [[i for i in range(1, len(graph) + 1)]]

    return graph, clusters


def test(graph_f, cut_f=None, also_test_full_graph=False):
    graph, cuts = graph_and_cut_to_numpy(graph_f, cut_f)
    #graph, cuts = graph_and_cut_to_numpy("graphs/whitaker3.graph", "old_results/whitaker3.graph.part.94.chaco")
    subgs = []

    for clidx, cut in enumerate(cuts):

        print("Start working on cluster;", clidx)
        if len(cut) == 1:
            print("Cut len is 1, continue")
            continue

        v_set = set()
        for el in cut:
            v_set.add(int(el))

        #print(v_set)
        n = len(v_set)

        subg = {k:[] for k in cut}
        #print("first element in cut", cut[0])
        for k in graph:
            for v in graph[k]:
                if v in v_set and k in v_set:
                    subg[k].append(v)


        rekey_dict    = {v:i for i, v in enumerate(sorted(list(subg.keys())))}
        rekey_reverse = {i:v for i, v in enumerate(sorted(list(subg.keys())))}

        success = True
        #new_subg = [[0 for _ in range(n)] for _ in range(n)]
        new_subg = sparse.dok_matrix((n, n))

        for k in subg: 

            if len(subg[k]) == 0:
                print("WARNING - disconnected cluster;", clidx, "number of nodes in subgraph is;", len(cut))
                success = False
                break

            for v in subg[k]:
                new_subg[rekey_dict[k], rekey_dict[v]] = 1
                new_subg[rekey_dict[v], rekey_dict[k]] = 1

                #new_subg[rekey_dict[k], rekey_dict[v]] += 1
                #new_subg[rekey_dict[v], rekey_dict[k]] += 1

        if not success:
            continue


        steps = test_convergence(new_subg, 0.01)

        print("Cluster", clidx, "of size: ", n, "took n steps to converge;", steps)

        import math
        print("steps/ log2 nodes;", steps/math.log2(n))

        #Scipy and numpy can't calculate eigenvalues quick
        
        n = len(graph)
    if also_test_full_graph: #n <= 5000
        trivial_subg = sparse.dok_matrix((n, n))
        for i, k in enumerate(graph):
            for j, _ in enumerate(graph[k]):
                #trivial_subg[i][j] = 1
                trivial_subg[i, j] = 1

        steps = test_convergence(trivial_subg, 0.01)
        print("Whole graph took n steps to converge:", steps)
        import math
        print("steps/ log2 nodes", steps/math.log2(n))
        


import glob


def run_decomp(graph_files, phi=0.01):
    for graph in graph_files:
        print("decomp on;", graph)

        #run decomposition on all graphs with time limit
        out_path = graph + ".out"
        #os.system
        #subprocess.check_call('time -p timeout 15m ./a.out  -S --G_phi=0.01 --H_phi=0.4 --vol=1 --h_ratio=0. -f $graph > "$out_path"', shell=True)
        try:
            subprocess.check_call("time -p timeout 15m ./a.out  -S --G_phi=" + str(phi) + " --H_phi=0.4 --vol=1 --h_ratio=0. -f " +  graph + " > " + out_path, shell=True)
        except Exception as e:
            print("Decomp failed with", e)
            #Too long time?
            continue

        #TODO return code
        if not glob.glob(graph + "cut.txt"):
            print("decomp on", graph, "did not finish in time")
            continue

        #how many clusters, k, did we get?
        with open(out_path) as f:
            lns = f.readlines()
            cluster_line = next(l for l in lns if "n clusters" in l)
            cluster_line.strip("\n")
            n_clusters = int(cluster_line.split(";")[1])

        if n_clusters <= 1:
            print("No cut found, continue")
            continue

        print("n clusters found", n_clusters)
        #run metis on k
        metis_stdio_path = graph + ".out.metis"


        try:
            subprocess.check_call("/usr/bin/gpmetis -ufactor=1000 " +  graph + " " + str(n_clusters) + " -contig >  " + metis_stdio_path, shell=True)
            metis_file = graph + ".part." + str(n_clusters)
            decomp_file = graph + "cut.txt"
            
            metis_partition_file_converter(metis_file, metis_file)


        except Exception as e:
            print("METIS failed", e)

        rw_graph      = graph + ".row_whole"
        rw_file_ours  = graph + ".rw_ours"
        rw_file_metis = graph + ".rw_metis"

        #TOFIX This should be saving to file!
        try:
            print("--- DECOMP OURS ---")
            test(graph, decomp_file)
            print("--- DECOMP METIS ---")
            test(graph, metis_file)  
        except Exception as e:
            print("Failed rw?", e)      

        bname = os.path.basename(graph).split(".")[0]
        os.system('mkdir -p results/"$bname"')
        for f in glob.glob("".join(graph.split(".")[:-1]) + "*"):
            if f != graph:
                os.system('mv $f results/"$bname"/')




In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import scipy.sparse as sparse
import torch
import scipy
import time
import datetime
import threading

#TODO check this for double links, self-loops
def test_convergence(matrix, threshold):

    n = matrix.shape[0]
    e = matrix.sum()

    print("n;", n)
    print("e;", e)

    #Number of in-links
    #Axii are inverted for some reason
    colsums = [i.item() for i in scipy.array((matrix.sum(axis=0))).T]
    uniform = np.array([0 for _ in range(n)], dtype=np.float64)

    #Stationary distribution is uniform normalized by in-links
    for i in range(n):
        uniform[i] = colsums[i]/e
    
    assert(0.99 <= uniform.sum() <= 1.01)
    assert(-0.01 < sum(colsums) - e < 0.01)

    #TODO seed the random start
    walk = np.array([1.] + [0 for _ in range(n - 1)], dtype=np.float64)
    walk = torch.DoubleTensor(walk)

    #Every node has positive in and out link
    sums = matrix.sum(axis=1)
    assert((sums != 0).any())
    sums = matrix.sum(axis=0)
    assert((sums != 0).any())

    rowsums = matrix.sum(axis=1)
    for i in range(n):
        matrix[i, i] = rowsums[i].item()
        cnt = rowsums[i] * 2
        matrix[i] /= cnt
        #assert(0.999 <= matrix[i].sum() <= 1.001)

    matrix = matrix.tocoo()   
    values = matrix.data
    indices = np.vstack((matrix.row, matrix.col))

    #Number of edges and self-loops
    assert(len(matrix.row) == len(matrix.col) == e + n)

    matrix = torch.sparse.DoubleTensor(torch.LongTensor(indices), torch.DoubleTensor(values), torch.Size(matrix.shape))

    all_ones = torch.DoubleTensor([1. for n in range(n)])

    assert(torch.allclose(all_ones, torch.sparse.sum(matrix, dim=1).to_dense()))

    uniform = torch.from_numpy(uniform)
    dist = torch.norm(walk - uniform, 1)

    steps = 0

    if torch.cuda.is_available():        
        matrix   = matrix.to("cuda")
        walk     = walk.to("cuda")
        uniform  = uniform.to("cuda")
        all_ones = all_ones.to("cuda")

    print("Start walk")

    step_lim = 10*n**3
    print_max = 15
    timeout_minutes = 10

    time_start = time.time()
    
    TIMER = 15.0

    def progress():
        p = threading.Timer(TIMER, lambda: progress())
        p.daemon=True
        p.start()
        print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), " - ", dist)

    #p = threading.Timer(TIMER, lambda: progress())
    #p.daemon=True
    #p.start()

    save_sum = dist.item()
    mm_transposed = matrix.transpose(1, 0)
    while dist > threshold and steps < step_lim: 
        walk = torch.sparse.mm(mm_transposed, walk.unsqueeze(-1)).squeeze(-1)
        #walk = torch.sparse.mm(matrix, walk.unsqueeze(-1)).squeeze(-1)

        assert(0.99  < torch.sum(walk)    < 1.01)
        assert(0.99  < torch.sum(uniform) < 1.01)
        assert(-0.02 < torch.sum(dist)    < 2.02)
        assert(-0.01 < torch.sum(walk - uniform) < 0.01)

        steps += 1
        assert(walk.shape == uniform.shape)
        dist = torch.norm(walk - uniform, 1)
        #Naive summation for higher precision
        #dist = 0.
        #np_walk = walk.tonumpy()
        #can be created just once
        #np_unif = unif.tonumpy()
        #dist = np.linalg.norm(np_walk - np_unif, 1)

        #for i in range(n):
        #    dist += (abs(walk[i] - uniform[i]))

        if steps % 5000 == 0:
            print("Dist", dist.item())

        """
        if (time.time() - time_start) > timeout_minutes:
            print("No convergence! ")
            return -1
        """

        assert(dist.item() <= save_sum)
        save_sum  = dist

    #p.cancel()
    print("dist;", dist)

    return steps

In [ ]:
import matplotlib.pyplot as plt

finan512_metis=!awk '{print NF}' results/finan512/finan512.graph.part.32
finan512_metis=[int(e) for e in finan512_metis]
finan512=!awk '{print NF}' results/finan512/finan512.graphcut.txt
finan512=[int(e) for e in finan512]

plt.hist([finan512, finan512_metis], bins=3, label=['Ours', 'METIS'])
plt.xlabel("Cluster volume")
plt.ylabel("Number of clusters")
plt.legend(loc='upper right')
plt.title("finan512")

plt.show()

In [ ]:
import matplotlib.pyplot as plt

t60k_metis=!awk '{print NF}' results/t60k/t60k.graph.part.60
t60k_metis=[int(e) for e in t60k_metis]
t60k=!awk '{print NF}' results/t60k/t60k.graphcut.txt
t60k=[int(e) for e in t60k]
plt.xlabel("Cluster volume")
plt.ylabel("Number of clusters")

plt.hist([t60k, t60k_metis], bins=20, label=['Ours', 'METIS'])
plt.legend(loc='upper right')
plt.title("t60k")
#plt.text(0.99,-0.1,'Volume of induced subgraphs', horizontalalignment='right')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

our   = []
metis = []

graph_paths = sorted(glob.glob("results/*"))
for graph in [os.path.basename(graph_path) for graph_path in graph_paths]:

    our_path  = glob.glob(os.path.join("results", graph) +  "/*graphcut*")
    our_path = our_path[0]

    metis_path = glob.glob(os.path.join("results", graph, "*.part.*"))[0]
    our_=!awk '{{print NF}}' $our_path
    #!awk '{{print NF}}' "$our_path"

    our_=[int(e) for e in our_]
    metis_=!awk '{{print NF}}' $metis_path

    metis_=[int(e) for e in metis_]
    our.append(our_)
    metis.append(metis_)


from math import log10

plt.figure()

our_box = plt.boxplot([[e  for e in lst] for lst in our],   positions=np.array(range(len(our)))  *2.0 -0.5, sym="", widths=0.5)
met_box = plt.boxplot([[e for e in lst] for lst in metis],  positions=np.array(range(len(metis)))*2.0 +0.5,  sym="", widths=0.5)

plt.setp(our_box['caps'],    color="C0")
plt.setp(our_box['medians'], color="C0")
plt.setp(our_box['boxes'],   color="C0")
plt.setp(our_box['whiskers'],color="C0")

plt.setp(met_box['caps'],    color="C1")
plt.setp(met_box['medians'], color="C1")
plt.setp(met_box['boxes'],   color="C1")
plt.setp(met_box['whiskers'],color="C1")

plt.yscale('log')

plt.plot([], c='C0', label='Ours')
plt.plot([], c='C1', label='METIS')

plt.legend(loc="lower left")

plt.xticks(range(0, 24, 2), ["3elt",  "4elt",  "add32",  "crack",  "cs4",  "cti",  "data",  "fe_4elt2",  "finan512",  "t60k",  "uk",  "whitaker3"], rotation=45, ha="right")

plt.ylabel("cluster size")
plt.xlim(-2, 24)
plt.ylim(0, 25000)

plt.tight_layout()



In [ ]:

from matplotlib.ticker import PercentFormatter

balances = !cat results/*/*.out | grep "vol graph"
balances = [float(e.split(";")[1]) for e in balances]
balances = [b for b in balances if b != -1]
print(balances)
print(len(balances))
print(max(balances))

plt.xlabel("Cut balance ")
plt.ylabel("Proportion of clusters")

plt.hist(balances, bins=10, label=['Vol(cut) / Vol(graph)'], weights=np.ones(len(balances)) / len(balances))
plt.legend(loc='upper right')
plt.title("Cut balance (all graphs)")
plt.text(0.99,-0.1,'Here cut refers to the side with higher volume.\n There were 171 cuts in total.', horizontalalignment='right')
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.show()

In [ ]:
import sys
import networkx as nx
import matplotlib.pyplot as plt

graph_file = "synthetic/margulis_25.graph"

G = nx.Graph()
with open(graph_file) as f_g:
    first_line = f_g.readline().split()
    n_nodes, n_edges = int(first_line[0]), int(first_line[1])
    iter = 0
    G.add_nodes_from([i for i in range(n_nodes)])
    for line in f_g:
        line_split = line.split()
        if len(line_split) <= 0:
            iter += 1
            continue
        base_node = int(line_split[0])
        for neighbor in line_split:
            G.add_edge(iter, int(neighbor) - 1)
        iter += 1

colors = [10 for _ in range(n_nodes)]

nx.draw_networkx(G, with_labels=False, node_color=colors, cmap=plt.cm.Blues, pos=nx.spring_layout(G))
plt.show()
nx.draw_circular(G, with_labels=False, node_color=colors, cmap=plt.cm.Blues)
plt.show()

In [ ]:
import glob
gls = list(glob.glob("results/*t60k*/*graphcut*"))
gls.sort()
print(gls)
#gls = gls[:2]

for gcut in gls:
    print(gcut)
    bname = os.path.basename(gcut)
    graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph  
    test(graph, gcut)

In [ ]:
import glob
gls = list(glob.glob("results/*add32*/*graphcut*"))

gls.sort()
print(gls)
#gls = gls[:2]

for gcut in gls:
    print(gcut)
    bname = os.path.basename(gcut)
    #graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph  
    graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph
    #test(graph, None) #Use whole graph as cut
    test(graph, gcut) #Use whole graph as cut



In [ ]:
#Only test graph

test("synthetic/barbell200-200.graph", None)


In [ ]:
#Only test graph

whitelist = ["3elt","4elt","add20","add32","bcsstk33","crack","cs4","cti","data","fe_4elt2","fe_pwt","fe_sphere","finan512","memplus","t60k","uk","whitaker3","wing_nodal"]
whitelist = ["t60k","uk","whitaker3","wing_nodal"]

files = []

for s in whitelist:
    files += glob.glob("graphs/" + s + ".graph")

files.sort()
for f in files:
    print(f)
    try:
        test(f, None)
    except AssertionError:
        print("walk failed")

In [ ]:
import glob
gls = list(glob.glob("graphs/*"))

gls = list(glob.glob("results/*/*graphcut*"))
gls2 = list(glob.glob("results/*/*.part.*"))
print(gls)
print(gls2)

print("START; OUR")
for g in gls:
    cont = False

    gp = "graphs/" + os.path.basename(g.split(".")[0]) + ".graph"
    #print(g, gp)
    #graph, clusters = graph_and_cut_to_numpy(gp, g)
    try:
        test(gp, g)
    except:
        print("???", "FAILED ON;", g)

print("DONE OUR; METIS")
for g in gls2:
    gp ="graphs/" + os.path.basename(g.split(".")[0]) + ".graph"
    print(g, gp)
    #graph, clusters = graph_and_cut_to_numpy(gp, g)
    try:
        test(gp, g)
    except:
        print("???", "FAILED ON;", g)

In [ ]:
import glob
gls = list(glob.glob("results/*/add32*part*"))
gls.sort()
#gls = gls[:2]
for gcut in gls:
    print(gcut)
    bname = os.path.basename(gcut)
    graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph
    test(graph, gcut)

In [ ]:
import glob
gls = list(glob.glob("results/add32/*graphcut*"))
gls.sort()
#gls = gls[:2]
for gcut in gls:
    print(gcut)
    bname = os.path.basename(gcut)
    graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph
    test(graph, gcut)

In [ ]:
import glob
gls = list(glob.glob("results/*/add32*part*"))
gls.sort()
#gls = gls[:2]
for gcut in gls:
    print(gcut)
    bname = os.path.basename(gcut)
    graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph
    test(graph, gcut)

In [ ]:

import glob
import subprocess

for phi in [0.01, 0.025, 0.05]:
    gls = list(glob.glob("graphs/whitaker3.graph"))
    run_decomp(gls, phi)
    print("phi=", phi)
    gls = glob.glob("graphs/whitaker3.graphcut.txt")
    for gcut in gls:
        print(gcut)
        bname = os.path.basename(gcut)
        graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph  (only works if k is single digit)
        test(graph, gcut)


In [ ]:
import matplotlib.pyplot as plt
size_steps_tuples = []
size_steps_tuples_metis = []
with open("walks_clusters.txt") as f:
    lines = f.read()
    _, lines_our, lines_metis= lines.split("OUR")
    print(lines_our[:100])
    print("###")
    print(lines_metis[:100])
    print("@@@")
    lines_our = lines_our.split("n, e ")
    lines_our = [line.split("\n") for line in lines_our]
    lines_metis = lines_metis.split("n, e ")
    lines_metis = [line.split("\n") for line in lines_metis]


    for i, graph_separated_output in enumerate(lines_our):
        #print("graph", i)

        counts = [l for l in graph_separated_output if "to converge" in l]
        #print(counts)
        if len(counts) <= 0:
            continue
        #print([line.split(" ") for line in counts])
        counts = [(int(l.split(" ")[5]), int(l.split(" ")[11])) for l in counts]

        #print("Len counts", len(counts), counts)
    
        size_steps_tuples += counts
    for i, graph_separated_output in enumerate(lines_metis):
        #print("graph", i)

        counts = [l for l in graph_separated_output if "to converge" in l]
        #print(counts)
        if len(counts) <= 0:
            continue
        #print([line.split(" ") for line in counts])
        counts = [(int(l.split(" ")[5]), int(l.split(" ")[11])) for l in counts]

        #print("Len counts", len(counts), counts)
    
        size_steps_tuples_metis += counts


from math import log2
#print(size_steps_tuples)
#print("n clusters", len(size_steps_tuples))

plt.xlim(5, 15) # = 15
plt.ylim(0, 30000) #  = 30000
a, b = np.polyfit(sorted([log2(v[0]) for v in size_steps_tuples], reverse=True), [v[1] for v in size_steps_tuples], 1)
plt.scatter(sorted([log2(v[0]) for v in size_steps_tuples], reverse=True), [v[1] for v in size_steps_tuples], label="slope: " + str(int(round(a))))
plt.xlabel("cluster size (log2)")
plt.ylabel("steps until convergence")
plt.legend(loc='upper right')
plt.title("Mixing rate on output clusters (ours)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
size_steps_tuples = []
size_steps_tuples_metis = []
with open("walks_clusters.txt") as f:
    lines = f.read()
    _, lines_our, lines_metis= lines.split("OUR")
    print(lines_our[:100])
    print("###")
    print(lines_metis[:100])
    print("@@@")
    lines_our = lines_our.split("n, e ")
    lines_our = [line.split("\n") for line in lines_our]
    lines_metis = lines_metis.split("n, e ")
    lines_metis = [line.split("\n") for line in lines_metis]


    for i, graph_separated_output in enumerate(lines_our):
        #print("graph", i)

        counts = [l for l in graph_separated_output if "to converge" in l]
        #print(counts)
        if len(counts) <= 0:
            continue
        #print([line.split(" ") for line in counts])
        counts = [(int(l.split(" ")[5]), int(l.split(" ")[11])) for l in counts]

        #print("Len counts", len(counts), counts)
    
        size_steps_tuples += counts
    for i, graph_separated_output in enumerate(lines_metis):
        #print("graph", i)

        counts = [l for l in graph_separated_output if "to converge" in l]
        #print(counts)
        if len(counts) <= 0:
            continue
        #print([line.split(" ") for line in counts])
        counts = [(int(l.split(" ")[5]), int(l.split(" ")[11])) for l in counts]

        #print("Len counts", len(counts), counts)
    
        size_steps_tuples_metis += counts

from math import log2
import numpy as np

a, b = np.polyfit(sorted([log2(v[0]) for v in size_steps_tuples_metis], reverse=True), [v[1] for v in size_steps_tuples_metis], 1)
print(a, b)
plt.scatter(sorted([log2(v[0]) for v in size_steps_tuples_metis], reverse=True), [v[1] for v in size_steps_tuples_metis], label="slope: " + str(int(round(a))), color='C1')
plt.xlabel("cluster size (log2)")
plt.ylabel("steps until convergence")
plt.legend(loc='upper right')
plt.title("Mixing rate on output clusters (METIS)")
plt.xlim(5, 15)
plt.ylim(0, 31000)



In [ ]:
with open("walks_graphs.txt") as f:
    lines = f.read()
    lines = lines.split(".graph\n")
    lines = [line.split("\n") for line in lines]
    print(lines[:10])